# IFT3700 - Science des donnees - Travail 1

## Travail présenté à M. Arnaud L'Heureux et M. Alain Tapp

## Auteurs :

### Ludovic Tuncay (20174139)

### Sobhan Mohammadpour (20166971)

### Thomas Lavend'homme (20165141)

### Marc-Antoine Dufresne Gagnon (20019871)

# Introduction


Pour ce projet, nous avons été introduits à l’ensemble de données MNIST, soit un ensemble de 60000 échantillons d’entrainement et 10000 échantillons de test. Chacun de ces échantillons correspondent à un tableau à 2 dimensions de 28 par 28 contenants les pixels d’images de chiffres écrits à la main. Le but de ce projet était d’améliorer la performance de différents algorithmes de partition sur cet ensemble en trouvant une meilleure notion de similarité et la comparer à la distance euclidienne. 

# Notions de similarité

## Similarité cosinus

### Méthodologie

Nous créons la matrice de similarité en considérant chaque point comme des vecteurs et calculant l'angle entre eux. Nous obtenons ce résultat en calculant le produit scalaire de chaque paire de points et divisant par le produit de leur norme. Cette métrique est rapide, mais ne fonctionne pas que  les ensembles de données qui ont des valeurs négatives. Lorsque nous avons beaucoup de données cette métrique devient résistant aux petits changements.

La fonction utilisée est la suivante :  

$$ cos(\theta) = \frac{\vec{a} \cdot \vec{b}}{||\vec{a}|| ||\vec{b}||} $$

Nous comparons dans ce rapport plusieurs méthodes, soit similarité cosinus, siamese net, hash, distance avec vecteurs et probabilitées avec réseau de neurones  dans le but de trouver la meilleure notion de similarité possible pour ce projet. Finalement, nous avons sélectionné la notion similarité cosinus. Nous élaborerons plus dans la section conclusion.



### Résultats

- KNN 1 neighbors
  - Accuracy : 0.97
  - v_measure : 0.92
- KMedoids 40 cluster
  - v_measure : 0.53
- Partition Binaire 100 cluster
  - v_measure : 0.64
- PCoA
  - Silhouette score: 0.00912

![](https://i.imgur.com/Mstqeim.png)


## Siamese net

### Méthodologie

Il faut trouver une transformation qui a la propriété suivante : si deux points de données sont près dans l'espace initial, alors leurs images respectives après la transformation seront proches, selon une distance euclidienne. On utilise un Convolutional Neutral Network afin de trouver une telle transformation. Ensuite, nous appliquons le `contrastive loss` sur la distance euclidienne pour entraîner le réseau.

### Résultats

- KNN 10 neighbors
  - Accuracy : 0.8595
  - v_measure : 0.727
- KMedoids 10 clusters
  - v_measure : 0.545
- Partition Binaire 25 clusters
  - v_measure : 0.50
- PCoA
  - Silhouette score: 0.030



![](https://i.imgur.com/T27lc91.png)
- Isomap
  - Silhouette score: 0.032

![](https://i.imgur.com/Ac5gL4L.png)


## Hash

### Méthodologie

Nous prenons une série de points aléatoires sur chacun des chiffres à comparer. On fait ensuite la norme et l'angle (par rapport à l'horizontale) du vecteur entre chaque point deux à deux. La transformation est une fonction de la longueur du vecteur, divisé par la moyenne de longueur de tous les vecteurs afin d'être insensible à l'échelle, ainsi qu'à l'angle trouvé, en soustrayant la moyenne de tous les angles, afin d'être insensible à la rotation originelle de l'image.

La fonction utilisée est la suivante :  

$A - \overline{A} + \frac{\pi}{2}$

où $A$ est l'angle du vecteur et $\overline{A}$ est la moyenne.

### Résultats

L'avantage de cette méthode est sa rapidité d'exécution. De plus, elle est facile à ajuster en fonction de la précision souhaitée.

Cela dit, cette méthode n'est pas très efficace. En effectuant KNN avec cette transformation, nous obtenons une précision d'environ 64,5%, alors que KNN avec la distance cosinus obtient une précision nettement meilleure d'environ 95%.

- KNN 100 neighbors
  - Accuracy : 0.70
  - v_measure : 0.53
- KMedoids 100 clusters
  - v_measure : 0.23
- Partition Binaire 20 clusters
  - v_measure : 0.2267
- PCoA
  - silhouette score: -0.0439



![](https://i.imgur.com/ND0G8FL.png)
- Isomap
  - silhouette score: -0.0557

![](https://i.imgur.com/CNeqWqw.png)


### Distance avec d'autre vecteurs

Comme la distance euclidienne entre les images fonctionne bien, nous pouvons utiliser la distance entre plusieurs stéréotypes comme une transformation. Cette méthode est flexible parce que nous pouvons choisir le nombre de dimension que nous avons (1000 dans notre cas). De plus, la méthode est rapide et simple à calculer. Grâce à la variété des données dans le jeu de données, de légères translations n'influencent pas les résultats, car il existera une autre image similaire dans les données initiales.

### Résultats

- KNN 10 neighbors
  - Accuracy : 0.9228
  - v_measure : 0.90
- KMedoids 20 clusters
  - v_measure : 0.45
- Partition Binaire 40 cluster
  - v_measure : 0.489
- PCoA
  - silhouette score: -0.02



![](https://i.imgur.com/sj1bYL3.png)
- Isomap
  - silhouette score: 0.03

![](https://i.imgur.com/9mtMHxE.png)


## Similarité avec un réseau de neurones

### Méthodologie

L'idée de cette similarité est d'utiliser un réseau de neurones pour déterminer si deux éléments sont similaires. Pour ce faire on va entrainer un MLP (mulit-layer perceptron) qui nous donnera en sortie un vecteur de probabilités. Ce vecteur correspondra aux probabilités que l'image appartienne aux différentes classes. Grâce à cela on définit la similarité entre deux images comme étant la distance euclidienne entre ces vecteurs de prédictions.
Pour utiliser cette distance nous avons deux possibilités :
1. Donner la fonction de similarité directement à l'algorithme (dans KNN  par exemple)
2. Transformer chaque donnée en entrée par son vecteur donné par le MLP. Ensuite, nous appliquons les algorithmes sur ces "nouveaux" points.
  
  + Puisque nous appliquons une distance euclidienne après, ces deux méthodes sont équivalentes (étant donné que les algorithmes utilisent la distance euclidienne par défaut)

Cette notion de similarité marche très bien et donne de meilleurs résultats que la distance euclidienne dans toutes les catégories (voir tableau ci-dessous).

### Résultats

        
  Temps pour entrainer le MLP |7min 29s|
  ----------------------------|--------|



modèles ->                  | Knn    |K-medoïds|Clustering|PCA      |Isomap   |PCoA |
----------------------------|:------:|:-------:|:--------:|:-------:|:-------:|:---:|
***distance euclidienne***  |        |         |          |         |         |
temps d'execution (fit)     |19.4 s* |11.9 s*  |48.6 s    |1.67 s   |43.4 s*  |23.5 s*
temps d'exécution (predict ou transform) |31.3 s*  |34.9ms   |No result |283 ms   |30.8 s*  |13.9 s*
v-measure score ou score silhouette|0.8948 (v measure)|0.3489 (v measure)|0.2346 (v measure)|0.0234 (silhouette)|0.0029 (silhouette)|0.0312 (silhouette)
***cette similarité***         |        |         |          |         |         |
temps d'execution (fit)     |62.1 ms*|8.15 s*  |3.76 s    |61.3 ms  |3.79 s*  |249 ms*
temps d'exécution (predict ou transform) |104 ms*  |4.79 ms  |No result |67.7 ms  |1.22 s* |11.9 s*
v-measure score ou score silhouette|0.9283 (v measure) |0.9399  (v measure) |0.9947 (v measure)|0.9564 (silhouette) |0.6118 (silhouette)  |0.9430 (silhouette)
* : fait sur un sous ensemble des données |    |          |         |         |

<sub><sup><sup><sub> Processeur sur lequel les temps sont mesurés : Processeur Intel Core i5 4 coeurs 2.3 GHz (8259U)</sub></sup></sup></sub>

1. PCA : On remarque que pour cette notion de similarité, la plupart des données sont rassemblées _en bas a gauche_. et d'autres données sont éparpillées sur le reste de l'espace. Ce résultat est intuitif, car dans l'espace transformé, les points similaires seront proche des vecteurs unités avec une variance très faible. Ceux-ci seront ainsi tous rassemblé dans une zone en appliquant PCA. Le reste des points sont plus difficiles à catégoriser. Par conséquent, ils seront plus éparpillés sur l'espace réduit final.  

 ![](https://imgur.com/lGmuSoZ.png) 
 
 ![](https://imgur.com/eKZrOKL.png)

  Pour confirmer cette notion on peut appliquer PCA pour qu'il ne reste qu'une seule dimension. Les données faciles à catégoriser seront rassemblées et les quelques points difficiles à catégoriser seront un peu plus loin dans l'espace. C'est effectivement ce qu'on observe :

 ![](https://imgur.com/dcmaRtx.png)


2. Isomap : 

 ![](https://imgur.com/LBDiV0x.png) 
 
 ![](https://imgur.com/ryjkvcg.png)



3. PCoA : 

 ![](https://imgur.com/UqInqyd.png)

 ![](https://imgur.com/Pvd25RV.png) 
 
 ![](https://imgur.com/qPHvjLY.png)



### Discussion

* Avantages :
  - Donne de très bons résultats
  - Une fois la transformation appliquée, le temps d'exécution devient beaucoup plus rapide.
  - Moins sensible aux petites modifications dans nos données (translations, etc ...)
* Désavantages :
  - La méthode est supervisée. En effet cette notion de similarité nécessite un ensemble d'entrainement labélisé pour fonctionner.
  - Le temps de calcul pris pour entrainer le réseau est assez long.

### Remarques :

- Cette méthode n'est en fait pas obligatoirement supervisée. En effet il est possible de prendre une autre méthode non supervisée pour obtenir ces vecteurs de probabilités, mais cela complique la tâche. La solution proposée est simplement une version plus facile à implémenter.
- Le temps de calcul est surtout pris pour l'entrainement du modèle, en effet une fois que nous "transformons nos points" le temps de calcul est plus rapide, car nous manipulons des données à 10 dimensions (versus 784).
- Cette notion de similarité est insensible aux modifications du moment que l'image transformée ressemble encore à un chiffre. Cette invariance est illustrée ci-dessous :  

distance             | 3 original | 3 avec modifications | autre 3 | 8 original
---------------------|:----------:|:--------------------:|:-------:|:---------:
3 original           |0           | 5.114e-06            |6.147e-40|1.414
3 avec modifications |5.114e-06   |0                     |5.114e-06|1.414
autre 3              |6.147e-40   |5.113e-06             |0        |1.414
8 original           |1.414       |1.414                 |1.414    |0

Voici les differentes images :




![](https://imgur.com/gHJP05z.png)

![](https://imgur.com/cUG8bkt.png)

![](https://imgur.com/vMQ0Aup.png) 

![](https://imgur.com/QIoB5X2.png)



  On remarque grâce au tableau que la forme influence peu le résultat. Les 3 sont similaires, mais ne le sont pas avec le 8. Cela confirme donc l'invariance de ma notion de similarité aux légères transformations.  
  <sup><sub>Remarque : les transformations appliquées ici sont des translations, des rotations et des étirements</sub></sup>

## Conclusion

Pour un jeu de données comme MNIST, puisque nous avons de bons labels (i.e. les données sont correctement labélisées), nous pouvons trouver de bonnes métriques. Par exemple en utilisant les probabilités avec un réseau de neurones (voir partie correspondante) qui donne de très bons résultats. Nous aurions aussi pu directement faire une transformation vers le label prédit (donc en une dimension). Cela donnerai une bonne partition mais l'idée de similarité serait plus abstaite. Cependant deux des gros désavantages de cette idée sont de devoir utiliser des données labelisées (i.e. methode supervisée) mais aussi le temps nécessaire à l'entrainement du modèle.

Si nous n'avions pas de labels, le nombre de données joue en notre faveur. En effet MNIST a une variété limitée. De légères transformations, translations ou rotations, n'influencent pas beaucoup les résultats finaux. Ce fait est même vrai pour la métrique "distance cosinus", qui n'est pas robuste à ce type de transformations. Ainsi, puisque nous avons accès à cette grande quantité de données, la distance cosinus est la métrique sans supervision avec laquelle nous avons expérimenté, qui est la plus rapide, la plus simple et qui au final a de très bons resultats.

Une autre bonne métrique est la distance avec vecteurs, qui est rapide et robuste sur les petites transformations.

Contrairement à nos hypothèses initiales, la métrique utilisant le hash n'a pas donné de bons résultats. Nous pensons que cette métrique est plus utile dans une situation avec des données plus éloignées et en moins grande quantité. Ce qui n'est pas le cas avec MNIST.

Les techniques de réduction de dimentionalités, comme PCoA et Isomap, n'ont pas générées de graphes avec des partitions distinctes. Nous voyons localement de bonnes partitions visiblement séparés, mais globalement nous n'obtenons pas de bons résultats de silhouettes (silhouette $\approx$ 0). Excepté la similarité avec réseau de neurones qui donne de très bons scores (voir tableau dans la partie correspondante).

